In [14]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 bs4 evaluate rouge_score

In [8]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from bs4 import BeautifulSoup

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
from huggingface_hub import login

login()

In [10]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "HuggingFaceH4/stack-exchange-preferences"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 8

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of fine-tuning examples - StackOverflow
num_examples_so = 2500

# Number of fine-tuning examples - Ed
num_examples_ed = 0

# Number of training epochs
num_train_epochs_so = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1 #num_train_epochs * num_examples_sf

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 5

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:
new_model_name = f"{model_name.split('/')[-1]}_so{num_examples_so}_ed{num_examples_ed}_LoRAr{lora_r}_lr{learning_rate}"
new_model_name

'Llama-2-7b-chat-hf_so2500_ed0_LoRAr16_lr0.0002'

## train model and look at loss curves

In [12]:
# Preprocess Data
def get_first_ans(example):
    ans_soup = BeautifulSoup(example['answers'][0]['text'], 'html.parser')
    ans_text = ans_soup.get_text()
    q_soup = BeautifulSoup(example['question'], 'html.parser')
    q_text = q_soup.get_text()
    example['answers'] = ans_text
    example['question'] = q_text
    qa_text = f"<s>[INST] {q_text} [/INST] {ans_text}</s>"
    # example['QA'] = qa_text
    return {'QA': qa_text}

from functools import partial
from datasets import Dataset

def iterable_ds_to_ds(iterable_ds):
    def gen_from_iterable_dataset(iterable_ds):
        yield from iterable_ds
    return Dataset.from_generator(partial(gen_from_iterable_dataset, iterable_ds), features=iterable_ds.features)

In [13]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, streaming=True, split='train')
dataset = dataset.map(
    get_first_ans,
    remove_columns=['qid', 'question', 'answers', 'date', 'metadata'])
dataset = iterable_ds_to_ds(dataset.take(num_examples_so))

# Load Ed dataset
ed_dataset = load_dataset('gperona/EdDataset', split=f'train[:{num_examples_ed}]')

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs_so,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="QA",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Set supervised fine-tuning parameters

# Train model
trainer.train()

# Save trained model
# trainer.model.save_pretrained(new_model_name)
# trainer.model.push_to_hub(new_model_name)
trainer.model.save_pretrained(new_model_name)

Resolving data files:   0%|          | 0/758 [00:00<?, ?it/s]

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/giuseppe/.local/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/giuseppe/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/giuseppe/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is reco

Step,Training Loss
5,2.570900
10,2.629600
15,2.663500
20,2.624800
25,2.587400
30,2.589700
35,2.560100
40,2.588000
45,2.541200
50,2.455700


In [14]:
del model
del trainer
import gc
gc.collect()
gc.collect()

0

In [15]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model_name)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
model.push_to_hub(new_model_name, use_temp_dir=False)
tokenizer.push_to_hub(new_model_name, use_temp_dir=False)

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/gperona/Llama-2-7b-chat-hf_so2500_ed0_LoRAr16_lr0.0002/commit/59ab434530af0ae0dec02dd9d2242f7009c21121', commit_message='Upload tokenizer', commit_description='', oid='59ab434530af0ae0dec02dd9d2242f7009c21121', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
%load_ext tensorboard
%tensorboard --logdir results/runs

Reusing TensorBoard on port 6006 (pid 20123), started 4:50:12 ago. (Use '!kill 20123' to kill it.)